In [2]:
import os

import jiant.utils.python.io as py_io
import jiant.proj.simple.runscript as simple_run
import jiant.scripts.download_data.runscript as downloader

In [3]:
TASK_NAME = "mnli"

# See https://huggingface.co/models for supported models
HF_PRETRAINED_MODEL_NAME = "microsoft/deberta-base"

In [4]:
MODEL_NAME = HF_PRETRAINED_MODEL_NAME.split("/")[-1]
RUN_NAME = f"simple_{TASK_NAME}_{MODEL_NAME}"
EXP_DIR = "/content/exp"
DATA_DIR = "/content/exp/tasks"

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(EXP_DIR, exist_ok=True)

In [4]:
downloader.download_data([TASK_NAME], DATA_DIR)

Downloading: 28.8kB [00:00, 14.4MB/s]                   
Downloading: 28.7kB [00:00, 14.3MB/s]                   


Downloading: 100%|██████████| 313M/313M [00:09<00:00, 32.7MB/s]


Dataset glue downloaded and prepared to C:\Users\Admin\.cache\huggingface\datasets\glue\mnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.
Downloaded and generated configs for 'mnli' (1/1)


In [5]:
args = simple_run.RunConfiguration(
    run_name=RUN_NAME,
    exp_dir=EXP_DIR,
    data_dir=DATA_DIR,
    hf_pretrained_model_name_or_path=HF_PRETRAINED_MODEL_NAME,
    tasks=TASK_NAME,
    train_batch_size=8,
    num_train_epochs=4
)
simple_run.run_simple(args)

Running from start
  jiant_task_container_config_path: /content/exp\run_configs\simple_mnli_deberta-base_config.json
  output_dir: /content/exp\runs\simple_mnli_deberta-base
  hf_pretrained_model_name_or_path: microsoft/deberta-base
  model_path: /content/exp\models\deberta\model\model.p
  model_config_path: /content/exp\models\deberta\model\config.json
  model_load_mode: from_transformers
  do_train: True
  do_val: True
  do_save: False
  do_save_last: False
  do_save_best: False
  write_val_preds: False
  write_test_preds: False
  eval_every_steps: 0
  save_every_steps: 0
  save_checkpoint_every_steps: 0
  no_improvements_for_n_evals: 0
  keep_checkpoint_when_done: False
  force_overwrite: False
  freeze_encoder: False
  seed: -1
  learning_rate: 1e-05
  adam_epsilon: 1e-08
  max_grad_norm: 1.0
  optimizer_type: adam
  no_cuda: False
  fp16: False
  fp16_opt_level: O1
  local_rank: -1
  server_ip: 
  server_port: 
device: cuda n_gpu: 1, distributed training: False, 16-bits training: 

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaModel: ['config', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'hidden_size': 768, 'hidden_dropout_prob': 0.1, 'vocab_size': 50265, 'model_arch': <ModelArchitectures.DEBERTA: 'deberta'>, 'classifier_type': 'linear'}


c:\Users\Admin\Desktop\research\prob-semantic-kg\jiant\proj\main\modeling\model_setup.py:186: UserWarning: The following weights were not loaded: dict_keys(['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias'])
  warnings.warn(
Training:   0%|          | 0/196352 [00:00<?, ?it/s]

No optimizer decay for:
  encoder.embeddings.LayerNorm.weight
  encoder.embeddings.LayerNorm.bias
  encoder.encoder.layer.0.attention.self.q_bias
  encoder.encoder.layer.0.attention.self.v_bias
  encoder.encoder.layer.0.attention.self.pos_q_proj.bias
  encoder.encoder.layer.0.attention.output.dense.bias
  encoder.encoder.layer.0.attention.output.LayerNorm.weight
  encoder.encoder.layer.0.attention.output.LayerNorm.bias
  encoder.encoder.layer.0.intermediate.dense.bias
  encoder.encoder.layer.0.output.dense.bias
  encoder.encoder.layer.0.output.LayerNorm.weight
  encoder.encoder.layer.0.output.LayerNorm.bias
  encoder.encoder.layer.1.attention.self.q_bias
  encoder.encoder.layer.1.attention.self.v_bias
  encoder.encoder.layer.1.attention.self.pos_q_proj.bias
  encoder.encoder.layer.1.attention.output.dense.bias
  encoder.encoder.layer.1.attention.output.LayerNorm.weight
  encoder.encoder.layer.1.attention.output.LayerNorm.bias
  encoder.encoder.layer.1.intermediate.dense.bias
  encoder.

Training:   0%|          | 113/196352 [00:27<13:28:34,  4.04it/s]


KeyboardInterrupt: 